*8.차원축소*

------

## 차원 축소
- 차원 축소를 시키면 일반적으로 __훈련 속도가 빨라짐__
- 어떤 경우에서는 성능이 좋아지는 경우도 있음  
- 차원을 축소시키면 어쩔 수 없이 __정보의 유실이 있음__
- 따라서 훈련속도가 빨라질 수는 있으나 성능이 떨어지는 경우도 생김
- 훈련속도와 성능을 적절히 고려하여 결과 도출 필요

----

## 차원의 저주
- 머신러닝에서 대부분의 경우 훈련 데이터와 특성이 엄청나게 큰 경향을 보임.
- 이는 학습을 느리게 할뿐 아니라 좋은 솔루션을 찾기 어렵게 만듦
- 이를 __차원의 저주__ 라고 함
> 고차원 공간에서 차원의 저주
> > - 고차원 공간은 대부분의 데이터가 서로 멀리 떨어져있음
> > - 이는 새로운 데이터도 멀리 떨어져 있을 가능성이 높다는 뜻임
> > - 이러한 경우 예측을 위해 모델이 저차원일 때보다 불안정해짐.
> > - 즉, __과적합의 위험이 커짐__

----

> 차원 축소
>> - 일반적으로 차원 축소는 피처 선택(Feature Selection)과 피처 추출(Feature Extraction)로 나뉨
>> - 여기서 PCA, LDA등은 피처 추출

|이름|특징|
|--|--|
|Feature Selection |Feature Selection은 원래 있던 피쳐들로부터 피쳐를 고름. 즉 값들의 변화가 없음|
|Feature Extraction |Feature Extraction은 원래는 없던 정보가 새로 생성됨. 즉, 원래 있던 피처들로부터 해당 피처들을 표현할 수 있는 정보들이 생성됨. |


## PCA(주성분분석)
> PCA 학습 과정
>> 1. 데이터를 원점으로 맞추어줌
>> 2. 데이터들의 분산이 최대가 되는 축(반대로 말하면 분산이 최대로 보존되는 축)을 찾고 데이터를 이 축에 사영시킴. -> 여기서 이 축을 첫번째 주성분(PC)라고 부름 
>> 3. 위에서 구했던 축과 직교하며 남은 분산이 최대가 되는 축을 찾고 이 축을 두번째 주성분라고 부름
>> 4. 축소를 원하는 차원까지 2,3을 반복

## LDA(Linear Discriminant Analysis)
> - PCA는 단순히 분산을 최대로하는 방식으로 차원을 축소했다면, LDA는 __데이터들을 잘 나눌 수 있도록 차원을 줄이는데에 중점__ 을 둔 알고리즘.
> - 즉, 분류에 적합한 알고리즘이며 __지도학습의 방식__
> - 분류모델을 사용하기 전에 LDA를 통해 차원을 줄이는데 좋음


---

## Code

### 타이타닉 데이터에 적용

In [1]:
# 모듈 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Waeing
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    
#matplotlib 한글깨짐 지원
import platform
from matplotlib import font_manager, rc
path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...')
rc('axes', unicode_minus=False)

- 데이터 불러오기

In [2]:
# 데이터 불러오기
path = './datasets/titanic/'
train = pd.read_csv(path+'train.csv', index_col='PassengerId') # `PassengerId` 열을 인덱스 열로 지정
test = pd.read_csv(path+'test.csv', index_col='PassengerId')

In [3]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


- 간단한 전처리

In [4]:
# encoder 함수 지정
def encoder(data, kind):
    # 범주형 변수와 수치형 변수를 분리
    cat_features = data.select_dtypes(include=['object']).columns.to_list()
    num_features = data.select_dtypes(exclude='object').columns.to_list() 
    if kind == 'onehot':
        # 원핫인코딩
        data_cat = pd.get_dummies(data[cat_features])
        data = pd.concat([data,data_cat],axis=1).drop(columns=[f for f in cat_features])
        return data
    elif kind == 'label':
        # 라벨인코딩
        for c in cat_features:
            data[c]=pd.Categorical(data[c]).codes
        return data
    else:
        print('choose "onehot"or "label"')
# 인코딩
train = encoder(train, 'onehot')
test = encoder(test, 'onehot')

In [5]:
# 간단한 결측값 처리
## train
train['Age'] = train['Age'].fillna(train['Age'].mode()[0])
## test
test['Age'] = test['Age'].fillna(test['Age'].mode()[0])
test['Fare'] = test['Fare'].fillna(test['Fare'].mode()[0])

- PCA in sklearn

In [6]:
# PCA
from sklearn.decomposition import PCA

def pca_data(data):
    pca = PCA()
    pca.fit(data) # pca 적용
    # 적절한 차원의 수를 찾기 위한 과정
    cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합 / 리스트의 형태
    d = np.argmax(cumsum >= 0.99) + 1 # 분산의 설명량이 99%이상 되는 인덱스를 cumsum에서 찾아줌 거기에 1을 더한것을 차원으로 지정 / 차원 갱신
    # 적용
    pca = PCA(n_components=d) # 분산의 설명량이 99% 이상 되는 차원으로 두번째 pca를 적용함    
    result = pd.DataFrame(pca.fit_transform(data))
    result.index = data.index # 인덱스를 맞춰줌
    return result
# 뭔가 조금 이상한 적용
train_pca = pca_data(train)
test_pca = pca_data(test)

train_pca = pd.concat([train_pca,train['Survived']],axis=1)

In [7]:
# 데이터를 학습과 검증으로 분리
x_train, x_valid, y_train, y_valid = train_test_split(train_pca.iloc[:,:-2], train_pca.iloc[:,2], test_size=0.2)

- 예측 -> 성은 그렇게 좋지는 않음 / 다들 이유 알잖아?

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

# 모델 선언
extra_tree = ExtraTreesClassifier(n_estimators=100, random_state=42)
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# fitting
extra_tree.fit(x_train, y_train)
random_forest.fit(x_train, y_train)

# 예측값
y_pred_extra_tree = extra_tree.predict(x_valid)
y_pred_random_forest = random_forest.predict(x_valid)

# 확인
print(f'extra_tree: {accuracy_score(y_valid, y_pred_extra_tree)}')
print(f'random_forest: {accuracy_score(y_valid, y_pred_random_forest)}')

extra_tree: 0.6536312849162011
random_forest: 0.6536312849162011


- LDA in sklearn -> 아주 유용할 것으로 생각했는데 n_components 1로밖에 안됨  
참고:  
[n_components 지정](https://stackoverflow.com/questions/26963454/lda-ignoring-n-components)

In [11]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(train.iloc[:,1:2], train.iloc[:,0])
train_lda = pd.DataFrame(lda.transform(train.iloc[:,1:2]))

In [12]:
train_lda

,0
0,0.878291
1,-1.662479
2,0.878291
3,-1.662479
4,0.878291
...,...
886,-0.392094
887,-1.662479
888,0.878291
889,-1.662479


## END